**clean up of the individual and the combined files**

In [1]:
#load all the relevant packages
import pandas as pd
import numpy as np
import os
import pycountry
import re
import datetime
import seaborn as sns
import matplotlib as mlt
import json

# load the dataframe

In [2]:
df = pd.read_csv('EU Sanctions map/eu_sanctions_map.csv',low_memory=False)

# get the number of listings per day

In [3]:
df['listing_count'] = 0  # Initialize the 'listing_count' column with zeros
df['listing_count'] = df.groupby('month')['id'].transform('nunique')
df

,Unnamed: 0,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,date_stamp,last_change,listing_date,delisting_date,month,listing_count
0,0,NK-2kLa63Qa2yXPxpd3zCFrwt,LegalEntity,Sovcombank,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,20220712,NaN,20220712,NaN,2022-07,12
1,1,NK-32NYYLd4iT2JDc2pukDRqY,LegalEntity,Bank Dabrabyt,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83
2,2,NK-3TSuSKtWtcuB3qBdLC8Y8d,Vessel,UL JI BONG 6,NaN,NaN,NaN,NaN,9114555,"""""",NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83
3,3,NK-4C4LEUKsLKwUnJmGeJjnp2,LegalEntity,ROSNEFT,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83
4,4,NK-4JYGfcjP3Frn24sYQ3ZXE2,Vessel,PAEK MA,NaN,NaN,NaN,NaN,9066978,DPRK vessel M/V PAEK MA was involved in ship-t...,NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,129,eu-sancmap-e51b08db944318e41b5d48fcea3d1e643ec...,LegalEntity,Pervyi Kanal,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,20230309,NaN,20230309,20230309.0,2023-03,10
130,130,eu-sancmap-f19cf43dd475a50ac0b6eb94fe005ed95ad...,Vessel,TONG SAN 2,NaN,NaN,NaN,NaN,445539000;8937675,"""""",NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83
131,131,eu-sancmap-fd1013b334b2efa28ab2ceda6fc53fc6bc2...,Vessel,PETREL 8,NaN,NaN,NaN,NaN,620233000;9562233,"""""",NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83
132,132,eu-sancmap-feb60d88b4a9c985444b47b0c297cba9ea4...,Vessel,AN SAN 1 Other information: DPRK tanker M/V AN...,NaN,NaN,NaN,NaN,7303803,"""""",NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83


# clean up country and iso columns

In [4]:
df.countries.unique()

array([nan])

In [5]:
iso =pd.read_csv('Clean/country-codes_csv.csv')
iso = iso.filter(['ISO3166-1-Alpha-3', 'MARC'])
iso=iso.rename(columns={'MARC':'countries','ISO3166-1-Alpha-3':'iso'})
display(iso)

,iso,countries
0,TWN,ch
1,AFG,af
2,ALB,aa
3,DZA,ae
4,ASM,as
...,...,...
245,ESH,ss
246,YEM,ye
247,ZMB,za
248,ZWE,rh


In [6]:
#check if there are any matches
matches_df = df[df['countries'].isin(iso['countries'])]
print(matches_df)

     Unnamed: 0                                                 id  \
0             0                          NK-2kLa63Qa2yXPxpd3zCFrwt   
1             1                          NK-32NYYLd4iT2JDc2pukDRqY   
2             2                          NK-3TSuSKtWtcuB3qBdLC8Y8d   
3             3                          NK-4C4LEUKsLKwUnJmGeJjnp2   
4             4                          NK-4JYGfcjP3Frn24sYQ3ZXE2   
..          ...                                                ...   
129         129  eu-sancmap-e51b08db944318e41b5d48fcea3d1e643ec...   
130         130  eu-sancmap-f19cf43dd475a50ac0b6eb94fe005ed95ad...   
131         131  eu-sancmap-fd1013b334b2efa28ab2ceda6fc53fc6bc2...   
132         132  eu-sancmap-feb60d88b4a9c985444b47b0c297cba9ea4...   
133         133  eu-sancmap-febf18c1754d6a3242b40757aabb5937ad0...   

          schema                                               name aliases  \
0    LegalEntity                                         Sovcombank     NaN   
1

In [7]:
# Split the 'marc_codes' column by semicolon into a list of codes
df['marc_codes'] = df['countries'].str.split(';')
matches_df = pd.merge(df,iso,how='inner')
matches_df

# Use the str.split with expand to create separate columns
df_expanded = matches_df['marc_codes'].apply(lambda x: pd.Series(x)).add_prefix('country_')

# Limit the number of expanded columns to 16
df_expanded = df_expanded.iloc[:, :17]

# Concatenate the new columns with the original DataFrame
df_result = pd.concat([matches_df, df_expanded], axis=1)

# Drop the original 'marc_codes' column if needed
df_result.drop(columns=['marc_codes'], inplace=True)

# Print or use the modified DataFrame as needed
df_result

AttributeError: Can only use .str accessor with string values!

In [ ]:
#check for completeness
df_result.iso.unique()

# clean up the sanctions column

In [9]:
#get all information in a list
list1=df.sanctions.values.tolist()
list1
#chunk the list to access all the information on the regimes
n=10
list(zip(*[iter(list1)]*n))
#save as json
with open("Clean/eu_map_regime.json", 'w') as f:
    json.dump(list1, f) 

with open("Clean/eu_map_regime.json", 'r') as f:
    sdn_sanction = json.load(f)

print(sdn_sanction)

['""', '""', '""', '""', 'DPRK vessel M/V PAEK MA was involved in ship-to-ship transfer operations for oil in mid-January 2018.', '""', '""', 'M/V DONG FENG 6 loaded DPRK coal at Hamhung, DPRK, on 11 July 2017, for export in violation of UN sanctions.', '""', '""', '""', '""', '""', '""', '""', 'M/V HUA FU loaded DPRK coal at Najin, DPRK, on 24 September 2017.', 'M/V HAO FAN 2 loaded DPRK coal at Nampo, DPRK, on 3 June 2017, for export in violation of UN sanctions.', 'M/V KOTI engaged in a ship-to-ship transfer, likely for oil, with M/V KUM UN SAN 3 on 9\xa0December 2017.', '""', '""', '""', '""', '""', '""', '""', ';M/V HAO FAN 6 loaded DPRK coal at Nampo, DPRK, on 27 August 2017.', '""', 'DPRK oil tanker M/V SAM MA 2 imported refined petroleum products in October, early November and mid-November 2017 through multiple ship-to-ship transfers.', '""', '""', '""', '""', '""', '""', 'The merchant vessel M/V SHANG YUAN BAO engaged in a ship-to-ship transfer, likely for oil, with UN-designa

# create year and month ids

In [10]:
df.month.unique()

array(['2022-07', '2022-05', '2023-03', '2022-06', '2022-11', '2023-07',
       '2023-08', '2023-04'], dtype=object)

In [11]:
# Convert the 'month' column to periods and factorize to get numerical month IDs
df['month_id'] = pd.to_datetime(df['month'], format='%Y-%m').dt.to_period('M').factorize()[0] + 1
df
# now i need to start earlier so i can match it with the other data (the IVs)

,Unnamed: 0,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,date_stamp,last_change,listing_date,delisting_date,month,listing_count,month_id
0,0,NK-2kLa63Qa2yXPxpd3zCFrwt,LegalEntity,Sovcombank,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,20220712,NaN,20220712,NaN,2022-07,12,1
1,1,NK-32NYYLd4iT2JDc2pukDRqY,LegalEntity,Bank Dabrabyt,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83,2
2,2,NK-3TSuSKtWtcuB3qBdLC8Y8d,Vessel,UL JI BONG 6,NaN,NaN,NaN,NaN,9114555,"""""",NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83,2
3,3,NK-4C4LEUKsLKwUnJmGeJjnp2,LegalEntity,ROSNEFT,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83,2
4,4,NK-4JYGfcjP3Frn24sYQ3ZXE2,Vessel,PAEK MA,NaN,NaN,NaN,NaN,9066978,DPRK vessel M/V PAEK MA was involved in ship-t...,NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,129,eu-sancmap-e51b08db944318e41b5d48fcea3d1e643ec...,LegalEntity,Pervyi Kanal,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,20230309,NaN,20230309,20230309.0,2023-03,10,3
130,130,eu-sancmap-f19cf43dd475a50ac0b6eb94fe005ed95ad...,Vessel,TONG SAN 2,NaN,NaN,NaN,NaN,445539000;8937675,"""""",NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83,2
131,131,eu-sancmap-fd1013b334b2efa28ab2ceda6fc53fc6bc2...,Vessel,PETREL 8,NaN,NaN,NaN,NaN,620233000;9562233,"""""",NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83,2
132,132,eu-sancmap-feb60d88b4a9c985444b47b0c297cba9ea4...,Vessel,AN SAN 1 Other information: DPRK tanker M/V AN...,NaN,NaN,NaN,NaN,7303803,"""""",NaN,NaN,EU Sanctions Map,20220510,NaN,20220510,NaN,2022-05,83,2


# save to csv

In [13]:
#save to csv
df.to_csv('Clean/eu_sanctions_map.csv')

# get descriptive statistics of the dataframe